# Approximate square roorts

I'm trying to approximate square roots for a integer number. The way to approximate the number is to represent the number to the binary system and represent each bit as a binary variable. The bit would either be 0 or 1. Unfortunately, the concept seems work, but the energy would somehow become negative and can't find the exact solution even the input is merely 4. In my opinion, this could be resulted by the intermediate variables generated by the `qubo` that causes the energy become negative.

In [1]:
from pyqubo import Array, Binary, Model
import numpy as np
import scipy as sp
import requests
import json
import re
import time
import yaml
from lib.util import *

import neal
sampler = neal.SimulatedAnnealingSampler()

In [2]:
def decode(configuration, dimension, precision, start_point):
    binary_code = create_binary_representated_vector(precision, start_point)
    nrows, ncols = dimension
    
    solution_dict = {}
    for key, value in configuration.items():
        if value:
            solution_dict[model.variables[int(key)]] = 1
        else:
            solution_dict[model.variables[int(key)]] = 0
            
    # binary_representation_columns is an array whose element are represented as a matrix
    # the matrix multiply binary_code would become a column
    binary_representation_columns = []
    for i in range(ncols):
        binary_representation_columns.append(np.zeros((nrows, precision)))

    for key, value in solution_dict.items():
        # get the name of column
        if not "*" in key:
            indexes = re.findall(r'(\d+)', key)
            indexes = [int(i) for i in indexes]
            col = indexes[0]
            binary_representation_columns[col][indexes[1]][indexes[2]] = value

    columns = []
    for mat in binary_representation_columns:
        columns.append(np.array((mat * np.matrix(binary_code).transpose()).transpose())[0])

    return np.matrix(np.column_stack(columns)), binary_representation_columns, binary_code

In [20]:
initial_values = { 0 : [[1, 0, None, False, True]] }

In [22]:
def setup_columns_config(variables:list, initial_values:dict):
    """Setup the initial configuration for each column
    
    In the approximation of vectors and matrices, it is general that representing a element of vector into 
    a set of bits time binary code.
    
    The function is used to setup the initial values for those bits
    
    Args:
        variables: a list of variables generated from your model.
        initial_values: a dictionary type of values which provides the initial values for each column
                        you can specify either 1 or True to represent your variable that is True and vice versa.
                        If you wouldn't like to provide the initial value for your variables, you can just provide 
                        None. For detail of use, please checkout the Example below.
    
    Returns:
        a dictionary of values to 
    
    Example:
        Here provides a basic example of inputs and output for your reference.
        There is only one number in a vector, and 5 bits used to represent the number; thus,
        it only requires a single column, and a row.
        
        The example want to set the following config of 0th column:
            0th bit: 1
            1st bit: 0
            2nd bit: skip
            3rd bit: 0
            4th bit: 1
        
        >>>model.variables
            ['col0[0][4]',
             'col0[0][3]',
             'col0[0][2]',
             'col0[0][0]',
             'col0[0][1]',
             'col0[0][4] * col0[0][3]',
             'col0[0][2] * col0[0][0]',
             'col0[0][4] * col0[0][1]',
             'col0[0][3] * col0[0][1]',
             'col0[0][1] * col0[0][4] * col0[0][3]']
         >>>initial_values
             {0: [[1, 0, None, False, True]]}
         >>>setup_columns_config(model.variables, initial_values)
    """
    config = {}
    for col_index in initial_values:
        array = initial_values[col_index]
        for i in range(len(array)):
            for j in range(len(array[i])):
                name = f"col{col_index}[{i}][{j}]"
                
                variable_index = variables.index(name)
                if array[i][j] == 1 or array[i][j] == True:
                    config[str(variable_index)] = True
                elif array[i][j] == 0 or array[i][j] == False:
                    config[str(variable_index)] = False
    return config

In [9]:
N = 4
precision = 5
starting_power = 1

X = generate_target_matrix((1, 1), precision, starting_power)[0, 0]
hamiltonian = (N - X*X)**2
start_time = time.time()

model = hamiltonian.compile()
qubo, offset = model.to_qubo()
matrix_term = get_matrix_term(qubo, model.variables)

end_time = time.time()
elapsed_time = end_time - start_time
print("Compile elapsed time: ", elapsed_time, " seconds")

Compile elapsed time:  0.0008060932159423828  seconds


## Use the DAU to approximate the number

In [10]:
problem_body = {}
DA_Solver = {}
DA_Solver["time_limit_sec"]= 2
DA_Solver["penalty_coef"]=10000
DA_Solver["num_run"] = 16
DA_Solver["num_group"] = 16
DA_Solver['gs_level'] = 50
DA_Solver['gs_cutoff'] = 80000
problem_body["fujitsuDA3"]=DA_Solver
problem_body["binary_polynomial"] = {'terms' : matrix_term}

initial_values = {
    0 : [[1, 0]]
}
initial_config = setup_columns_config(model.variables, initial_values)
# problem_body["fujitsuDA3"]['fixed_config'] = initial_config

response = post_solve(problem_body)
job_id = response['job_id']

{'message': 'success'}


In [14]:
solution = get_solution(job_id)
configuration = solution['qubo_solution']['solutions'][0]['configuration']
energy = solution['qubo_solution']['solutions'][0]['energy']
mat, bits, binary_code = decode(configuration, (1, 1), precision, starting_power)
number = mat[0, 0]
print("number : ", number)
print("error : ", (N - number*number)**2)
print("energy : ", energy)
delete_res = delete_job(job_id)

number :  2.0
error :  0.0
energy :  -16


## Use the simulated annealing algorithm to approximate the root

In [15]:
bqm = model.to_bqm()
sampleset = sampler.sample(bqm, num_reads=1000)
decoded_samples = model.decode_sampleset(sampleset)
best_sample = min(decoded_samples, key=lambda x: x.energy)
sample_sol = best_sample.sample

bits = np.zeros((1, precision))
for key, val in sample_sol.items():
    if not "*" in key:
#         print(key)
        indexes = re.findall("\[(\d+)\]", key)
        indexes = [int(i) for i in indexes]
#         print(indexes, val)
        bits[indexes[0]][indexes[1]] = val
    
binary_code = np.matrix(create_binary_representated_vector(precision, starting_power)).transpose()
bits = np.matrix(bits)
number = bits*binary_code
print("number : ", number[0, 0])
print("energy : ", best_sample.energy)

number :  2.0
energy :  0.0
